This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [2]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'T8vgAsnQkxGs5TzuC-Hs'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [1]:
# First, import the relevant modules
import requests


In [5]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
headers = {
    "start_date": "2014-01-01",
    "end_date"  : "2014-01-02",
    "api_key"   : API_KEY
}
sample_request = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json", params=headers)

In [6]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
sample_request.json()

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2014-01-01',
  'end_date': '2014-01-02',
  'frequency': 'daily',
  'data': [['2014-01-02',
    24.24,
    24.76,
    23.77,
    24.18,
    None,
    55735.0,
    1355658.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

First, we call the API to get the data for the year requested. 

In [7]:
headers = {
    "start_date": "2017-01-01",
    "end_date"  : "2017-12-31",
    "api_key"   : API_KEY
}
data_raw = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json", params=headers)

Then, we change the JSON into a dictionary using the `.json` method.

In [9]:
dat_dict = data_raw.json()

It might help for us to extract the inner data list of lists from the dictionary for easy working, ane enumerate the headers for easy reference.

In [37]:
dat_list = dat_dict["dataset_data"]["data"]
col_names = dict(enumerate(dat_dict["dataset_data"]["column_names"]))
col_names

{0: 'Date',
 1: 'Open',
 2: 'High',
 3: 'Low',
 4: 'Close',
 5: 'Change',
 6: 'Traded Volume',
 7: 'Turnover',
 8: 'Last Price of the Day',
 9: 'Daily Traded Units',
 10: 'Daily Turnover'}

We notice from our quick glance above that the Open price is found in column 1 of the data. Furthermore, we notice that sometimes we have missing values represented as None, so that also needs to be taken into account. Thus, we can write:

In [28]:
opens = [entry[1] for entry in dat_list if entry[1]]
(min(opens), max(opens))

(34.0, 53.11)

We can use a similar method to calculate the largest change in one day, by calculating the changes of every day in the set and then taking the maximum of the list. 

In [31]:
delta = [entry[2] - entry[3] for entry in dat_list if (entry[2] and entry[3])]
max(delta)

2.8100000000000023

This strategy can also be extended to find the largest change between any two days, although some infrastructure needs to be built first. 

In [50]:
class comp_pair:
    def __init__(self, comp, **kwargs):
        self.comp = comp
        self.lookup = kwargs
    def __gt__(self, oth):
        return self.comp > oth.comp
    def __lt__(self, oth):
        return self.comp < oth.comp
    def __eq__(self, oth):
        return self.comp == oth.comp
    def __str__(self):
        out = str(self.comp)
        for key, value in self.lookup.items():
            out = out + "; " + str(key) + ": " + str(value)
        return out
    
close_with_date = [comp_pair(entry[4], date=entry[0]) for entry in dat_list if entry[4]]
(str(min(close_with_date)), str(max(close_with_date)))

('34.06; date: 2017-01-23', '53.09; date: 2017-12-13')

In [51]:
53.09 - 34.06

19.03

From the above we can see that the close price differences were greatest between 2017-12-13 and 2017-01-23 with a difference of 19.03.

Using the same extraction technique, we can calculate the mean using some basic Python functions:

In [55]:
volume = [entry[6] for entry in dat_list if entry[6]]
sum(volume)/len(volume)

89124.33725490196

And lastly, we can calculate the median trading volume by writing a custom median function:

In [75]:
def median(lst):
    def quickselect(lst, k):
        if len(lst) == 1:
            assert k == 0
            return lst[0]
        
        pivot = lst[int(len(lst)/2)]
        lows = [i for i in lst if i < pivot]
        high = [i for i in lst if i > pivot]
        same = [i for i in lst if i == pivot]
        
        if k < len(lows):
            return quickselect(lows, k)
        elif k < len(lows) + len(same):
            return pivot
        else:
            return quickselect(high, k-len(lows)-len(same))
        
    if len(lst) % 2 == 1:
        return quickselect(lst, (len(lst)-1)/2)
    else:
        return 0.5*(quickselect(lst, len(lst)/2-1)+
                    quickselect(lst, len(lst)/2))

In [76]:
median(volume)

76286.0